In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from google.colab import files

!pip install -q tensorflow-hub
import tensorflow_hub as hub

url1 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv'
url2 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv'

train_df = pd.read_csv(url1)
test_df = pd.read_csv(url2)

train_df.head(5)

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [2]:
train_df = train_df.drop(['Unnamed: 0', 'id', 'keyword', 'location', 'text'],axis = 1)
train_target_df = train_df.pop('target')
test_df = test_df.drop(['Unnamed: 0', 'id', 'keyword', 'location', 'text'],axis = 1)

train_df.shape

(7613, 14)

In [3]:
train_df.head()

,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [4]:
train_df['has_mentions'] = train_df['has_mentions'].astype(int)
train_df['has_link'] = train_df['has_link'].astype(int)
train_df['contains_countries'] = train_df['contains_countries'].astype(int)
train_df['contains_cities'] = train_df['contains_cities'].astype(int)
train_df.head()

,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,69,13,0,0,0,0,0,0,0,0,0,7,0,0
1,38,7,1,0,0,0,0,0,0,0,0,3,1,0
2,133,22,0,0,0,0,0,0,0,0,0,13,0,0
3,65,8,1,0,0,0,0,0,0,0,0,2,0,0
4,88,16,1,0,0,0,0,0,0,0,0,10,0,0


In [17]:
# ARMAMOS EL MODELO
model = tf.keras.Sequential()
model.add(tf.keras.layers.LayerNormalization(input_shape = tuple([train_df.shape[1]])))
model.add(tf.keras.layers.Dense(14, activation = 'sigmoid'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(14, activation = 'sigmoid'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_6 (Layer (None, 14)                28        
_________________________________________________________________
dense_16 (Dense)             (None, 14)                210       
_________________________________________________________________
dropout_2 (Dropout)          (None, 14)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 14)                210       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 15        
Total params: 463
Trainable params: 463
Non-trainable params: 0
_________________________________________________________________


In [18]:
# COMPILAMOS Y ENTRENAMOS
train_features_list = np.asarray(train_df)
train_target_list = np.asarray(train_target_df)

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_features_list, train_target_list, batch_size=512,
                    epochs = 900,
                    verbose = 1)

Epoch 1/900
15/15 [==============================] - 0s 2ms/step - loss: 0.7324 - accuracy: 0.5703
Epoch 2/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6989 - accuracy: 0.5703
Epoch 3/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6879 - accuracy: 0.5703
Epoch 4/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6842 - accuracy: 0.5703
Epoch 5/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6863 - accuracy: 0.5703
Epoch 6/900
15/15 [==============================] - 0s 3ms/step - loss: 0.6854 - accuracy: 0.5703
Epoch 7/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6858 - accuracy: 0.5703
Epoch 8/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6845 - accuracy: 0.5703
Epoch 9/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6849 - accuracy: 0.5703
Epoch 10/900
15/15 [==============================] - 0s 2ms/step - loss: 0.6854 - accuracy: 0.5703
Epoch 11/

In [8]:
# EVALUAMOS CON EL SET DE VALIDACION
loss, precision = model.evaluate(train_features_list, train_target_list)
print('Precisión: %.2f' % (precision*100))

238/238 [==============================] - 0s 1ms/step - loss: 0.6475 - accuracy: 0.5703
Precisión: 57.03


In [ ]:
# ARMAMOS LA PREDICCIÓN
test_text_list = np.asarray(test_df['text'])
prediction = (model.predict(test_text_list) > 0.5).astype("int32")
prediction

In [ ]:
# EXPORTAMOS EL CSV EN EL FORMATO QUE PIDE KAGGLE
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')